<a href="https://colab.research.google.com/github/ssudhakar165/colab/blob/main/yolo_yaml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ultralytics/yolov3  # clone repo


Cloning into 'yolov3'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 9711 (delta 0), reused 2 (delta 0), pack-reused 9708
Receiving objects: 100% (9711/9711), 8.24 MiB | 686.00 KiB/s, done.
Resolving deltas: 100% (6622/6622), done.



# download command/URL (optional)


# ___train and val data as 1) directory: path/images/, 2) file: path/images.txt, or 3) list: [path1/images/, path2/images/]
train: /content/yolov3/images/train  # 128 images


val: /content/yolov3/images/val  # 128 images

# __number of classes
nc: 8

# __class names
names: ["Person","Chair","Car","Bottle","Pottleplant","Dog","Boat","Bird"]








In [2]:
%mv /content/coco128.yaml /content/yolov3/data/coco128.yaml

after correcting upload and move to as above

In [3]:
%cd yolov3
!pip install -r requirements.txt  # install dependencies

/content/yolov3
     |████████████████████████████████| 645kB 4.3MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip '/content/drive/MyDrive/pascal.zip'

Archive:  /content/drive/MyDrive/pascal.zip
  inflating: train.csv               
  inflating: val.csv                 
  inflating: train_images/2007_000027.jpg  
  inflating: train_images/2007_000032.jpg  
  inflating: train_images/2007_000033.jpg  
  inflating: train_images/2007_000039.jpg  
  inflating: train_images/2007_000042.jpg  
  inflating: train_images/2007_000061.jpg  
  inflating: train_images/2007_000063.jpg  
  inflating: train_images/2007_000068.jpg  
  inflating: train_images/2007_000121.jpg  
  inflating: train_images/2007_000123.jpg  
  inflating: train_images/2007_000129.jpg  
  inflating: train_images/2007_000170.jpg  
  inflating: train_images/2007_000175.jpg  
  inflating: train_images/2007_000187.jpg  
  inflating: train_images/2007_000241.jpg  
  inflating: train_images/2007_000243.jpg  
  inflating: train_images/2007_000250.jpg  
  inflating: train_images/2007_000256.jpg  
  inflating: train_images/2007_000272.jpg  
  inflating: train_images/2007_000323.jpg  


In [6]:
import numpy as np
import pandas as pd

from PIL import Image

In [7]:
train= pd.read_csv('train.csv')

In [8]:
train['class']= train['class'].replace({'person':0,'chair': 1, 'car':2, 'bottle':3, 'pottedplant':4,'dog':5,'boat':6, 'bird':7})

Please number it from 0 to 7 , not 1 to 8.

In [9]:
train['class'].value_counts()

0              1749
1               298
2               275
3               162
4               156
5               126
6               124
7               102
sheep            99
tvmonitor        98
cat              98
sofa             93
cow              92
motorbike        87
horse            86
bicycle          80
aeroplane        76
diningtable      71
train            63
bus              57
Name: class, dtype: int64

In [10]:
train= train.loc[ (train['class']==1)|(train['class']==2)|(train['class']==3)|(train['class']==4)|(train['class']==5)|(train['class']==6)|(train['class']==7)|(train['class']==0)]

In [11]:
train['class'].value_counts()

0    1749
1     298
2     275
3     162
4     156
5     126
6     124
7     102
Name: class, dtype: int64

In [12]:
!mkdir labels
!mkdir images
!mkdir train
!mkdir val

In [13]:
for image_name in train.filename.unique():

    # separate labels for each image
    image_metadata = train[train.filename == image_name].copy()

    # x-center, y-center, height and width for bounding box
    image_metadata['x_center'] = (image_metadata['xmax'] + image_metadata['xmin']) / 2
    image_metadata['y_center'] = (image_metadata['ymax'] + image_metadata['ymin']) / 2
    image_metadata['width'] = (image_metadata['xmax'] - image_metadata['xmin'])
    image_metadata['height'] = (image_metadata['ymax'] - image_metadata['ymin'])

    im_width, im_height = Image.open('train_images/' + image_name).size

    # normalizing bounding box values
    image_metadata['x_center'] = image_metadata['x_center']/im_width
    image_metadata['width'] = image_metadata['width']/im_width
    image_metadata['y_center'] = image_metadata['y_center']/im_height
    image_metadata['height'] = image_metadata['height']/im_height

    image_metadata[['class', 'x_center', 'y_center', 'width', 'height']].to_csv('train/' + image_name.split('.')[0] + '.txt', index=False, header=None, sep=" ")

In [14]:
pascalDet_train = train.filename.apply(lambda x: '/content/yolov3/images/train/' + x).unique()
np.savetxt("pascalDet_train.txt", pascalDet_train, fmt="%s")

In [15]:
!cat pascalDet_train.txt

/content/yolov3/images/train/2007_000027.jpg
/content/yolov3/images/train/2007_000032.jpg
/content/yolov3/images/train/2007_000061.jpg
/content/yolov3/images/train/2007_000063.jpg
/content/yolov3/images/train/2007_000068.jpg
/content/yolov3/images/train/2007_000129.jpg
/content/yolov3/images/train/2007_000170.jpg
/content/yolov3/images/train/2007_000241.jpg
/content/yolov3/images/train/2007_000250.jpg
/content/yolov3/images/train/2007_000272.jpg
/content/yolov3/images/train/2007_000323.jpg
/content/yolov3/images/train/2007_000346.jpg
/content/yolov3/images/train/2007_000363.jpg
/content/yolov3/images/train/2007_000364.jpg
/content/yolov3/images/train/2007_000392.jpg
/content/yolov3/images/train/2007_000423.jpg
/content/yolov3/images/train/2007_000452.jpg
/content/yolov3/images/train/2007_000480.jpg
/content/yolov3/images/train/2007_000504.jpg
/content/yolov3/images/train/2007_000515.jpg
/content/yolov3/images/train/2007_000529.jpg
/content/yolov3/images/train/2007_000559.jpg
/content/y

In [16]:
val= pd.read_csv('val.csv')
pascalDet_val = train.filename.apply(lambda x: '/content/yolov3/images/val/' + x).unique()
np.savetxt("pascalDet_val.txt", pascalDet_val, fmt="%s")

Keep train and val images same. usually train_images to see the performance

In [17]:
!cat pascalDet_val.txt

/content/yolov3/images/val/2007_000027.jpg
/content/yolov3/images/val/2007_000032.jpg
/content/yolov3/images/val/2007_000061.jpg
/content/yolov3/images/val/2007_000063.jpg
/content/yolov3/images/val/2007_000068.jpg
/content/yolov3/images/val/2007_000129.jpg
/content/yolov3/images/val/2007_000170.jpg
/content/yolov3/images/val/2007_000241.jpg
/content/yolov3/images/val/2007_000250.jpg
/content/yolov3/images/val/2007_000272.jpg
/content/yolov3/images/val/2007_000323.jpg
/content/yolov3/images/val/2007_000346.jpg
/content/yolov3/images/val/2007_000363.jpg
/content/yolov3/images/val/2007_000364.jpg
/content/yolov3/images/val/2007_000392.jpg
/content/yolov3/images/val/2007_000423.jpg
/content/yolov3/images/val/2007_000452.jpg
/content/yolov3/images/val/2007_000480.jpg
/content/yolov3/images/val/2007_000504.jpg
/content/yolov3/images/val/2007_000515.jpg
/content/yolov3/images/val/2007_000529.jpg
/content/yolov3/images/val/2007_000559.jpg
/content/yolov3/images/val/2007_000629.jpg
/content/yo

In [18]:
with open('pascalDet.names', 'w') as f:
    f.write('Person\nChair\nCar\nBottle\nPottleplant\nDog\nBoat\nBird')


In [19]:
with open('pascalDet.data', 'w') as f:
    f.write('classes=8\ntrain=/content/yolov3/pascalDet_train.txt\nvalid=/content/yolov3/pascalDet_val.txt\nnames=/content/yolov3/pascalDet.names')

In [20]:
!cat pascalDet.data

classes=8
train=/content/yolov3/pascalDet_train.txt
valid=/content/yolov3/pascalDet_val.txt
names=/content/yolov3/pascalDet.names

move images of train and val into images=

Images

Train

Val

Labels

Train

Lab

The above is important for processing

In [21]:
! python train.py --img 320 --batch 8 --epochs 5 --data coco128.yaml --weights yolov3.pt

github: up to date with https://github.com/ultralytics/yolov3 ✅
YOLOv3 v9.1-4-gcf5db95 torch 1.7.0+cu101 CUDA:0 (Tesla K80, 11441.1875MB)

Namespace(adam=False, batch_size=8, bucket='', cache_images=False, cfg='', data='./data/coco128.yaml', device='', epochs=5, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[320, 320], local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp', single_cls=False, sync_bn=False, total_batch_size=8, weights='yolov3.pt', workers=8, world_size=1)
wandb: Install Weights & Biases for YOLOv3 logging with 'pip install wandb' (recommended)
Start Tensorboard with "tensorboard --logdir runs/train", view at http://localhost:6006/
2021-01-28 08:51:10.490566: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library

some time you may get error. pl change image size and run. Pl use GPU, cpu is very slow

Pl go to  RUN folder and check for result. TXT and Result .png.
You will have some batches performance.